In [ ]:
library(Seurat)
library(Matrix)
library(reticulate)
library(tools)

library(data.table)
library(spacexr)
library(readr)
library(ggplot2)
library(tidyverse)

In [ ]:
getwd()

In [ ]:
bin_name = 'bin20'
name = 'Y01804JC_9'
group = 'annotation'
spatial = paste0('/data/work/Y01804JC_9/Y01804JC_9_10w_SCT.rds') 

output = paste0('/data/work/HSJ/1x1/RCTD/')

if (!dir.exists(output)) dir.create(output, recursive = TRUE); setwd(output)

In [ ]:
colorlist = c("#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941",
             "#006FA6", "#A30059", "#FFE4E1", "#0000A6", "#63FFAC",
             "#B79762", "#004D43", "#8FB0FF", "#997D87", "#5A0007",
             "#809693", "#1B4400", "#4FC601", "#3B5DFF", "#FF2F80",
             "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9",
             "#B903AA", "#DDEFFF", "#7B4F4B", "#A1C299", "#0AA6D8",
             "#00A087FF", "#4DBBD5FF", "#E64B35FF", "#3C5488FF", "#F38400",
             "#A1CAF1", "#C2B280", "#848482", "#E68FAC", "#0067A5",
             "#F99379", "#604E97", "#F6A600", "#B3446C", "#DCD300",
             "#882D17", "#8DB600", "#654522", "#E25822", "#2B3D26",
             "#191970", "#000080","#6495ED", "#1E90FF", "#00BFFF", 
             "#00FFFF", "#FF1493","#FF00FF", "#A020F0", "#63B8FF", 
             "#008B8B", "#54FF9F","#00FF00", "#76EE00", "#FFF68F",
             "#FF69B4", "#8A2BE2", "#00FF7F", "#FFD700", "#FF6347", "#4682B4",
             "#00FA9A", "#20B2AA", "#87CEEB", "#778899", "#B0C4DE", "#ADFF2F",
             "#FF69B4", "#8A2BE2", "#00FF7F", "#FFD700", "#FF6347", "#4682B4",
             "#00FA9A", "#20B2AA", "#87CEEB", "#778899", "#B0C4DE", "#ADFF2F",
             "#FF69B4", "#8A2BE2", "#00FF7F", "#FFD700", "#FF6347", "#4682B4",
             "#00FA9A", "#20B2AA", "#87CEEB", "#778899", "#B0C4DE", "#ADFF2F")

## 导入空间组数据
SeuObj = readRDS(spatial)
print("SeuObj read successfully")

## 导入单细胞数据
sc = readRDS(paste0("/data/work/HSJ/1x1/S1_B03404C4.rds")) 
# meta = read.csv(metadata,row.names=1)
# sc@meta.data = meta[colnames(sc),]
# sc = subset(sc,sample == bin_name)

group = group
cell_low <- table(sc@meta.data[[group]])[table(sc@meta.data[[group]]) <25] %>% names()
Idents(sc) <- sc@meta.data[[group]]
sc <- subset(sc ,idents = cell_low, invert = TRUE)
sc@meta.data[[group]] <- as.factor(sc@meta.data[[group]])
sc=subset(sc, downsample = 500) # sc@meta.data[[group]] 列 每个 cell type 保留最多 500 个细胞 downsampl 提提速

print(table(sc@meta.data[[group]]))
print("sc read successfully！")

In [ ]:
###提取空间组数据的信息

# 直接赋值，保持 dgCMatrix 稀疏格式
exp_spatial = SeuObj@assays$Spatial@counts

# exp_spatial=as.matrix(SeuObj@assays$Spatial@counts)
# exp_spatial=as.data.frame(exp_spatial)  # RCTD 支持 matrix/dgCMatrix，不需要 dataframe

SeuObj$coor_y = SeuObj@images[[names(SeuObj@images)]]@coordinates$row
SeuObj$coor_x = SeuObj@images[[names(SeuObj@images)]]@coordinates$col
coord <- data.frame(stereo_1 = SeuObj$coor_x, stereo_2 = SeuObj$coor_y)
#SeuObj[["stereo"]] <- SeuratObject::CreateDimReducObject(embeddings = as.matrix(coord), key = "stereo_", assay = "Spatial")

row <- SeuObj$coor_y
col <- SeuObj$coor_x
coord_spatial <- data.frame(row,col)
rownames(coord_spatial)<- Cells(SeuObj)
nUMI_spatial=SeuObj@meta.data[,"nCount_Spatial"]
names(nUMI_spatial)=rownames(SeuObj@meta.data)

###提取单细胞的数据
# 直接赋值，保持 dgCMatrix 稀疏格式
exp_sc = sc@assays$RNA@counts
# exp_sc=as.matrix(sc@assays$RNA@counts)
# exp_sc=as.data.frame(exp_sc) # RCTD 支持 matrix/dgCMatrix，不需要 dataframe

celltype_sc=sc@meta.data[[group]]
celltype_sc <- gsub("[/\\\\ ]", "_", celltype_sc)  # 替换 / \ 和空格
celltype_sc <- as.factor(celltype_sc)
names(celltype_sc)=rownames(sc@meta.data)
celltype_sc=as.factor(celltype_sc)
nUMI_sc=sc@meta.data[,"nCount_RNA"]
names(nUMI_sc)=rownames(sc@meta.data)

###构建RCTD的数据数据（puck和reference）
reference <- Reference(exp_sc, celltype_sc, nUMI_sc)
puck <- SpatialRNA(coord_spatial, exp_spatial, nUMI_spatial)
print("data have praperad successfully！")

###清除内存
rm(sc)
gc()




In [ ]:

myRCTD <- create.RCTD(puck, reference, max_cores = 8, UMI_min=10, UMI_min_sigma = 100)
myRCTD <- run.RCTD(myRCTD, doublet_mode = 'multi' )

saveRDS(myRCTD, file = paste0(bin_name, "_myRCTD_multi.rds"))
print("RCTD successfully！")

In [13]:
str(myRCTD)

Formal class 'RCTD' [package "spacexr"] with 9 slots
  ..@ spatialRNA        :Formal class 'SpatialRNA' [package "spacexr"] with 3 slots
  .. .. ..@ coords:'data.frame':	100000 obs. of  2 variables:
  .. .. .. ..$ x: num [1:100000] 1062 953 348 930 231 ...
  .. .. .. ..$ y: num [1:100000] 258 69 945 1004 157 ...
  .. .. ..@ counts:Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. ..@ i       : int [1:27957655] 9 13 17 19 22 23 25 27 28 29 ...
  .. .. .. .. ..@ p       : int [1:100001] 0 390 577 889 1248 1374 1707 2116 2255 2575 ...
  .. .. .. .. ..@ Dim     : int [1:2] 2734 100000
  .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. ..$ : chr [1:2734] "CERS6" "TBXAS1" "FOXN2" "USP3" ...
  .. .. .. .. .. ..$ : chr [1:100000] "BIN.1247994" "BIN.1119621" "BIN.409017" "BIN.1093508" ...
  .. .. .. .. ..@ x       : num [1:27957655] 1 1 3 1 1 1 2 1 2 1 ...
  .. .. .. .. ..@ factors : list()
  .. .. ..@ nUMI  : Named num [1:100000] 1343 609 1193 1355 419 ...
  .. .. .. ..

In [15]:
library(tidyverse)
library(ggplot2)

print("正在重新处理 Multi-mode 结果...")

# --- 1. 修正：安全地获取 Barcodes 和 坐标 ---
# 我们直接从 spatialRNA 对象中获取坐标和行名，而不是从 results 列表中取名
coords <- myRCTD@spatialRNA@coords
barcodes <- rownames(coords)

# 双重保险：如果 coords 没有行名，尝试从 counts 矩阵获取列名
if (is.null(barcodes)) {
  barcodes <- colnames(myRCTD@spatialRNA@counts)
  # 确保 coords 行数和 barcodes 长度一致，防止意外
  if(nrow(coords) == length(barcodes)) {
    rownames(coords) <- barcodes
  }
}

print(paste0("检测到 Barcodes 数量: ", length(barcodes)))

# --- 2. 提取权重信息 ---
results_list <- myRCTD@results

# 提取 all_weights
# 这里的 sapply 会生成一个 矩阵 (列数=细胞类型数, 行数=Spot数)
weights_matrix <- t(sapply(results_list, function(x) x$all_weights))

# 强制赋予行名，确保对应关系
rownames(weights_matrix) <- barcodes

# 归一化权重
norm_weights <- sweep(weights_matrix, 1, rowSums(weights_matrix), '/')
norm_weights[is.na(norm_weights)] <- 0

# --- 3. 提取分类信息 ---
# 找出每个 Spot 权重最大的细胞类型
major_type_index <- max.col(norm_weights, ties.method = "first")
first_type <- colnames(norm_weights)[major_type_index]

# 提取包含的细胞类型数量
n_cell_types <- sapply(results_list, function(x) length(x$cell_type_list))

# --- 4. 构建最终表格 ---
# 确保所有向量长度一致
if(length(barcodes) != length(first_type)) {
  stop("错误：Barcodes 数量与结果数量不匹配，请检查数据完整性。")
}

results_df <- data.frame(
  barcode = barcodes,
  x = coords$x,
  y = coords$y,
  first_type = first_type,
  n_cell_types = n_cell_types,
  stringsAsFactors = FALSE
)

# 合并权重数据
full_results_csv <- cbind(results_df, norm_weights)

print("数据处理成功！准备保存...")

# --- 5. 保存 ---

# 保存 CSV
write.csv(full_results_csv, file = paste0(bin_name, "_RCTD_multi_results.csv"), row.names = FALSE)
print("CSV 已保存。")



[1] "正在重新处理 Multi-mode 结果..."
[1] "检测到 Barcodes 数量: 100000"
[1] "数据处理成功！准备保存..."
[1] "CSV 已保存。"


In [16]:
head(full_results_csv)

,barcode,x,y,first_type,n_cell_types,CD14_CD16_monocyte,CD14_monocyte,CD16_monocyte,CD16_NK_cell,Cytotoxic_CD8+_T_cell,Dendritic_cell,Effector_memory_CD8+_T_cell,Megakaryocyte,Memory_B_cell,Memory_CD4+_T_cell,Naive_B_cell,Naive_CD4_T_cell,Naive_CD8+_T_cells,NKT_cell,Regulatory_T_cell
,<chr>,<dbl>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BIN.1247994,BIN.1247994,1062,258,CD14_monocyte,2,0.0001436266,0.3280594577,0.149299506,0.0001436266,0.0001436266,0.0001436266,0.0001436266,0.0001436266,0.0189516066,0.0312214944,0.0001436266,0.0058319787,0.2891848355,0.1763021079,0.0001436266
BIN.1119621,BIN.1119621,953,69,Naive_CD8+_T_cells,1,0.1739653475,0.0002197851,0.057338257,0.0002197851,0.0701856409,0.0002197851,0.0002197851,0.0276015997,0.1653726921,0.0002197851,0.0002197851,0.0002197851,0.2728863493,0.0578715224,0.1732400951
BIN.409017,BIN.409017,348,945,Naive_CD8+_T_cells,1,0.0001625905,0.1835127599,0.004181564,0.0001625905,0.0001625905,0.0001625905,0.1166460665,0.0177581433,0.0001625905,0.1732865375,0.0001625905,0.1911296690,0.3121845356,0.0001625905,0.0001625905
BIN.1093508,BIN.1093508,930,1004,Memory_CD4+_T_cell,1,0.0002492335,0.2527878219,0.181490290,0.0799473994,0.0668707322,0.1083977100,0.0002492335,0.0122033105,0.0002492335,0.2963088678,0.0002492335,0.0002492335,0.0002492335,0.0002492335,0.0002492335
BIN.270637,BIN.270637,231,157,Naive_CD4_T_cell,1,0.0001745293,0.1057286835,0.206769261,0.0135165763,0.1458859009,0.0001745293,0.0001745293,0.0001745293,0.0001745293,0.0001745293,0.1322499128,0.3942789020,0.0001745293,0.0001745293,0.0001745293
BIN.801900,BIN.801900,682,1044,Memory_CD4+_T_cell,2,0.0009672768,0.2271291125,0.246411009,0.0001624333,0.0001624333,0.1239902331,0.0001624333,0.0064621144,0.0001624333,0.3613584437,0.0001624333,0.0323823438,0.0001624333,0.0001624333,0.0001624333


In [17]:
# --- 6. 绘图 ---                     
                       
# 绘图 1: 主要细胞类型
cell_name <- sort(unique(results_df$first_type))
my_colors <- colorlist[1:length(cell_name)]
names(my_colors) <- cell_name

p1 <- ggplot(results_df, aes(x=x, y=y, fill=first_type)) +
  geom_tile() + 
  coord_fixed() +
  labs(title = "Dominant Cell Type (Multi Mode)", fill = "Cell Type") +
  theme_void() +
  theme(
    panel.background = element_rect(fill = "black"),
    plot.background = element_rect(fill = "black"),
    text = element_text(color = "white"),
    plot.title = element_text(color = "white", hjust = 0.5),
    legend.text = element_text(color = "white")
  ) +
  scale_fill_manual(values = my_colors)

ggsave(paste0(bin_name, "-multi_cell_type.pdf"), plot=p1, height = 12, width = 14)

# 绘图 2: 细胞类型数量分布
p2 <- ggplot(results_df, aes(x=x, y=y, fill=as.factor(n_cell_types))) +
  geom_tile() + 
  coord_fixed() +
  labs(title = "Number of Cell Types per Spot", fill = "Count") +
  theme_void() +
  theme(
    panel.background = element_rect(fill = "black"),
    plot.background = element_rect(fill = "black"),
    text = element_text(color = "white"),
    plot.title = element_text(color = "white", hjust = 0.5),
    legend.text = element_text(color = "white")
  ) +
  scale_fill_brewer(palette = "Set1")

ggsave(paste0(bin_name, "-multi_n_types.pdf"), plot=p2, height = 10, width = 12)

print("绘图完成！")

[1] "绘图完成！"
